In [ ]:
import numpy as np
from numpy import pi as π
import scipy
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tqdm

## Coupled oscillators

**Why are we doing this:** This notebook will look at systems of many coupled harmonic oscillators, which, in the limit of infinitely many oscillators, gives us the shallow water or seismic wave equations.
We can work out the hard parts of discretizing wave-type equations in time using this example.

Here we'll in more detail at coupled systems of harmonic oscillators.
In the previous notebook, we looked at how to simulate a system of four oscillators in a linear chain.
This demo will look at what happens when we look at larger systems.
The displacement $q$ obeys the 2nd-order ODE
$$\ddot q + D\;\text{diag}(\omega)^2\;D^*\,q = 0$$
where $D$ is the incidence matrix and $\omega$ is the vector of fundamental frequencies.
Recall that the incidence matrix has shape `(num_weights, num_springs)`.
If spring $k$ connects weights $i_1$ and $i_2$, then $D_{i_1,k} = +1$ and $D_{i_2,k} = -1$.
We get a different matrix if we flip $i_1$ and $i_2$, but this doesn't matter for the end result.

Fill in the body of the routine below to compute the incidence matrix.
The input `springs` is a numpy array of shape `(num_springs, 2)`, with each row corresponding to one spring.
The total number of weights can be found by computing the maximum value of `springs` and adding 1 (remember that Python is 0-indexed).

In [ ]:
def make_incidence_matrix(springs):
    ...

The code below will make the incidence matrix for a linear chain of 24 weights connected by springs.

In [ ]:
num_weights = 24
springs = np.array([(i, i + 1) for i in range(num_weights - 1)])
num_springs = springs.shape[0]
D = make_incidence_matrix(springs)

You can use an `assert` call when you want to check that something is true.

In [ ]:
assert D.shape[0] == num_weights
assert np.sum(np.abs(np.ones(num_weights) @ D)) == 0

Before, we reduced the oscillator system to first order by introducing a new variable $v = \dot q$.
There's more than one way that we could do this however.
For example, if we instead take
$$\dot q = D\,\text{diag}(\omega) v,$$
then we can rewrite the system as
$$\left[\begin{matrix}\dot q \\ \dot v\end{matrix}\right] + \left[\begin{matrix}0 & -D\,\text{diag}(\omega) \\ \text{diag}(\omega)\,D^* & 0\end{matrix}\right]\left[\begin{matrix} q \\ v\end{matrix}\right] = 0.$$
(Check that this is equivalent to the 2nd-order problem I wrote above if you're not sure.)
Write some code below to compute the matrix for this system and store it in a variable `A`.
You'll need to use `np.block`.
Also, I've write $0$ twice to denote matrices whose entries are all zero.
This is a little bit of an abuse of notation because each of those zeros has a different number of rows and columns.
So you'll need to make two different size zero matrices.
It's worth working out ahead of time what the sizes of all the block matrices are.
Remember that the shape of $D$ is `(num_weights, num_springs)` and that you might have different numbers of weights and springs.

In [ ]:
A = ...

Make some initial displacement of the weights in a vector `q` and an initial "velocity" in a vector `v`.
You can make the initial velocity all 0s to start.
Then use np.concatenate to stack them into one vector `z`.

In [ ]:
q = ...
v = ...
z = ...

Let's examine the eigenvalues and eigenvectors of `A`.

In [ ]:
λ, Q = scipy.linalg.eig(A)

Print out the imaginary parts of the eigenvalues of $A$.
Notice anything about them?

Now print out the sum of the absolute values of the real parts of the eigenvalues.
Does the result make sense?

Now write the simulation loop.
This should be similar to your solution for the random walk notebook.
Use a final time of $8\pi$ and 256 timesteps.
Compute the propagator matrix $G = \exp(-dt\cdot A)$, make an array `zs` to hold the solution values at all the desired time intervals, and write a loop to fill this array.
Use the same order as in the random walk notebook, i.e. the time index comes first.

In [ ]:
zs = ...
# Your code here

If you did everything right, this should show you a pretty movie.

In [ ]:
%%capture
fig, ax = plt.subplots()
x = np.array(list(range(num_weights)))
qs = zs[:, :num_weights]
ax.set_ylim((qs.min(), qs.max()))
points = ax.scatter(x, qs[0])
def animate(q):
    points.set_offsets(np.column_stack((x, q)))
animation = FuncAnimation(fig, animate, zs[:, :num_weights], interval=1e3/30)

In [ ]:
HTML(animation.to_jshtml())

The code below will plot the kinetic, potential, and total energy.
If everything worked right, the total energy should be constant.

In [ ]:
qs = zs[:, :num_weights]
vs = zs[:, num_weights:]

kinetic_energies = np.array([0.5 * np.inner(v, v) for v in vs])
potential_energies = np.array([0.5 * np.inner(q, q) for q in qs])

In [ ]:
fig, ax = plt.subplots()
ax.plot(kinetic_energies, label="kinetic")
ax.plot(potential_energies, label="potential")
ax.plot(kinetic_energies + potential_energies, label="total")
ax.legend();